
## P7: Résolvez des problèmes en utilisant des algorithmes en Python #1


Repartir des essais de découverte précédent pour améliorer la calcul brute force au mieux.
Si la contrainte est toujours du style budget maximum, il est inutile de conserver les combinaisons dont le cout est > BUDGET.
Cela n'en devient pas pour autant de la programmation dynamique, réservée à l'algo suivant ;
la programmation dynamique évitant de recalculer des sous-arbres déja parcourus.


* Caculer simultanément à l'explosion combinatoire.
Dans les essais de découverte, j'ai dans un 1er temps explosé toutes les combinaisons possibles puis ensuite seulement calculé et limité au sous-ensemble de budget <= BUDGET.
Un calcul simultané permettrait la vérification de contrainte non atteinte et la réduction de l'ensemble des solutions possibles.

* Structure de donnée.
Pour "porter" les combinaisons une liste est envisageable avec pour chercher les données (cout, profit) de chaque action un dictionnaire semble efficace.



### 1. Préparation des données



In [1]:
import csv as csv
import re as re
import time



In [2]:
try:
    import big_o
except ModuleNotFoundError:
    print('cherche',ModuleNotFoundError)
    !pip install --upgrade pip
    !pip install big_o
    import big_o

In [3]:
# constant
#FILE = "data/p7-20-shares.csv" 
#FIELDNAMES = ['name', 'cost', 'profit_share', 'profit']

FILE = "data/dataset1_Python+P7" 
FIELDNAMES = ['name', 'cost', 'profit']
BUDGET = 50000

### En complément de Big_O, mesure du temps

Si la mesure du temps d'execution est dépendante du type de PC, des processus en cours d'execution etc., elle permet une approximation de l'efficacité  des algorithme choisis complémentaire à celle de la notation big_o.

Suivant le conseil de [Marina Mele](http://www.marinamele.com/author/marina-melegmail-com) dans [7 tips to Time Python scripts and control Memory & CPU usage](https://www.marinamele.com/7-tips-to-time-python-scripts-and-control-memory-and-cpu-usage)


In [4]:
 def fn_timer(function):
        
#    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.perf_counter_ns()
        result = function(*args, **kwargs)
        t1 = time.perf_counter_ns()
        print ("Total time running %s: %s nanoseconds" %
               (function.__name__, str(t1-t0))
               )
        return result
    return function_timer

In [5]:
# nettoyer les données des fichiers en entrée
def clean_char(texte: str) -> str:
    """ on ne conserve que les caractères lisibles 
    les lettres, chiffres, ponctuations décimales et signes
    les valeurs negatives sont acceptées, du point de vue profit.
    """
    texte_propre = re.sub(r"[^a-zA-Z0-9\-\.\,\+]", "", texte.replace(',','.'))
    return texte_propre

### Amélioration des données

Sur python le travail avec float est plus couteux qu'en entier.
Une solution est alors de multiplier par 100 cout, budget et profit car cela ne change pas
le résultat.


In [6]:
""" lecture, nettoyage et chargement en dict.
    les non valeurs NaN sont rejetées.
"""
action_dict = {}
try:
    with open(FILE, "r", newline='', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file, fieldnames=FIELDNAMES, 
                                    delimiter=';', doublequote=False)
        # skip the header
        next(csv_reader)
        for idx, line in enumerate(csv_reader):
            clean_data = True
            if line[FIELDNAMES[0]] != "":
                cle = clean_char(line[FIELDNAMES[0]])
            else:
                print(f" line {idx} had missing share name; dropped.")
                clean_data = False
            if line[FIELDNAMES[1]] != "":
                cout = 100 * int(clean_char(line[FIELDNAMES[1]]))
            else:
                print(f" line {idx} had missing cost data; dropped.")
                clean_data = False
            if line[FIELDNAMES[3]] != "":
                gain = int(100 * float(clean_char(line[FIELDNAMES[3]])))
            else:
                print(f" line {idx} had missing profit data; dropped.")
                clean_data = False
            if (gain < 0) or (cout < 0):
                print(f" line {idx} had negative value; accepted but pls check.")
            if (clean_data):
                action_dict[cle] = (cout, gain)
except FileNotFoundError:
    print(f" fichier non trouvé, Merci de vérifier son nom {file_name} : {FileNotFoundError}")            
except IOError:
    print(f" une erreur est survenue à l'écriture du fichier {file_name} : {IOError}")            


In [7]:
action_dict
                

{'Action-1': (2000, 100),
 'Action-2': (3000, 300),
 'Action-3': (5000, 750),
 'Action-4': (7000, 1400),
 'Action-5': (6000, 1019),
 'Action-6': (8000, 2000),
 'Action-7': (2200, 154),
 'Action-8': (2600, 286),
 'Action-9': (4800, 624),
 'Action-10': (3400, 918),
 'Action-11': (4200, 714),
 'Action-12': (11000, 990),
 'Action-13': (3800, 874),
 'Action-14': (1400, 14),
 'Action-15': (1800, 54),
 'Action-16': (800, 64),
 'Action-17': (400, 48),
 'Action-18': (1000, 140),
 'Action-19': (2400, 504),
 'Action-20': (11400, 2052)}

### 2. Calcul brute force.

$
{n \choose k}={\frac {n!}{k!(n-k)!}}.


### 2.1 Estimation simple du nombre de calcul maximum.

entendre : Non réduit à la contrainte BUDGET.

In [8]:
def number_combi(n: int, k: int) -> int:
    def factorielle(x:int) -> int:
        if x <=1:
            return 1
        else:
            return (x * factorielle(x-1))
    top = factorielle(n)
    bot = factorielle(k) * factorielle(n-k)
    if bot > 0:
        return top/bot
    else:
        raise Error

In [9]:
denombrement = False
if denombrement:
    N = len(action_dict.keys())
    number_of_combi = 0
    for K in range(N):
        number_of_combi += number_combi(N, K)
    print(number_of_combi)   


Nous sommes prévenu, il y a environ 1 million de combinaison d'un portefeuille de 20 actions!
Le calcul des combinaisons va être long!!  
  


#### 2.3 Fonction combinatoire proposée.

In [10]:
# initialize
actions_list = [([x],y[0],y[1]) for x,y in action_dict.items()]
actions_list

[(['Action-1'], 2000, 100),
 (['Action-2'], 3000, 300),
 (['Action-3'], 5000, 750),
 (['Action-4'], 7000, 1400),
 (['Action-5'], 6000, 1019),
 (['Action-6'], 8000, 2000),
 (['Action-7'], 2200, 154),
 (['Action-8'], 2600, 286),
 (['Action-9'], 4800, 624),
 (['Action-10'], 3400, 918),
 (['Action-11'], 4200, 714),
 (['Action-12'], 11000, 990),
 (['Action-13'], 3800, 874),
 (['Action-14'], 1400, 14),
 (['Action-15'], 1800, 54),
 (['Action-16'], 800, 64),
 (['Action-17'], 400, 48),
 (['Action-18'], 1000, 140),
 (['Action-19'], 2400, 504),
 (['Action-20'], 11400, 2052)]

In [11]:
def explode_iter(elements, length):
    """ global action_dict to fetch cost & profit 
    elements would be list(tuple(list, cost, profit))
    """
  
    # pour le 1er élément de la combinaison, il peut prendre toute valeur de la liste
    for i in range(len(elements)):
        # singelton -> on rapporte des t-uple de 1 avec le 1er élément lui-même
        if length == 1:
            yield (elements[i][0] , elements[i][1], elements[i][2])
        else:
            # on prend la liste réduite à partir de l'élément suivant i et 
            # on en cherche les combinaisons de longueur -1
            
            #print('sous-liste:',elements[i+1:len(elements)])
            #print('à i:',i)
            for next1 in explode_iter(elements[i+1:len(elements)], length-1):

                #print('next1:',next1)
                #print('elements[0][1]',elements[i][1])
                #print('next1[0]:',next1[0])
                # et on retourne la liste précédent  plus la liste de l'appel suivant
                cout = elements[i][1] + next1[1]
                profit = elements[i][2] + next1[2]
                # optimisation
                if cout <= BUDGET:
                    yield (elements[i][0] + next1[0],
                           cout,
                           profit)
# toutes les combinaisons de k éléments de la liste l

In [12]:
list(explode_iter(actions_list, 1))

[(['Action-1'], 2000, 100),
 (['Action-2'], 3000, 300),
 (['Action-3'], 5000, 750),
 (['Action-4'], 7000, 1400),
 (['Action-5'], 6000, 1019),
 (['Action-6'], 8000, 2000),
 (['Action-7'], 2200, 154),
 (['Action-8'], 2600, 286),
 (['Action-9'], 4800, 624),
 (['Action-10'], 3400, 918),
 (['Action-11'], 4200, 714),
 (['Action-12'], 11000, 990),
 (['Action-13'], 3800, 874),
 (['Action-14'], 1400, 14),
 (['Action-15'], 1800, 54),
 (['Action-16'], 800, 64),
 (['Action-17'], 400, 48),
 (['Action-18'], 1000, 140),
 (['Action-19'], 2400, 504),
 (['Action-20'], 11400, 2052)]

In [13]:
@fn_timer
def algo_force_brute(portefeuille: list) -> list:
    liste_output = []
    for largeur in range(1,len(portefeuille)+1):
        liste_output.extend(list(explode_iter(portefeuille, largeur)))
    return liste_output

In [14]:
explosion_force_brute = algo_force_brute(actions_list)


Total time running algo_force_brute: 13020773700 nanoseconds


In [15]:
#explosion_force_brute

Résultat 


Total time running algo_force_brute: 13138267100 nanoseconds


Le print de l'explosion combinatoire est lourd en ressource.

`IOPub data rate exceeded.
`The Jupyter server will temporarily stop sending output
`to the client in order to avoid crashing it.
`To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



 
 
 #### Etape 3 déterminer l'élément à plus haut profit pour un budget <= 50000
 
 Un tri puis une recherche dichotomique?
 ou
 Une recherche non dichotomique dans une liste non préalablement triée??
 ou le parcours complet de la liste une fois
 

In [16]:
## tri par profit puis cout décroissant
explosion_force_brute.sort(key = lambda sub : sub[2] ,reverse = True)

In [17]:
explosion_force_brute[0]

(['Action-4',
  'Action-5',
  'Action-6',
  'Action-8',
  'Action-10',
  'Action-11',
  'Action-13',
  'Action-18',
  'Action-19',
  'Action-20'],
 49800,
 9907)

In [18]:
meilleur_profit = 0
meilleur_budget = 0
meilleur_combinaison = []
for courant in explosion_force_brute:
    if (courant[2] > meilleur_profit) & (courant[1] < BUDGET):
        meilleur_profit = courant[2]
        meilleur_budget = courant[1]
        meilleur_combinaison = courant[0]


In [19]:
print(meilleur_profit, meilleur_budget, meilleur_combinaison)

9907 49800 ['Action-4', 'Action-5', 'Action-6', 'Action-8', 'Action-10', 'Action-11', 'Action-13', 'Action-18', 'Action-19', 'Action-20']


9907 49800 ['Action-4', 'Action-5', 'Action-6', 'Action-8', 'Action-10', 'Action-11', 'Action-13', 'Action-18', 'Action-19', 'Action-20']

